# Capstone Project - The Battle of the Neighborhoods

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# Introduction/Business Problem
## 1. A description of the problem and a discussion of the background

In this project I will compare neiberhoods of two big cities in Europe, Amsterdam and Berlin.

My target audience are people that know one city pretty well and are wondering which neighborhood of the other city is most similar to the one they like.

They could use this data to choose location of their hotel or apartment when visiting or moving to the other city. 

Criteria that I will take into the consideration are, Population density, area, venues nearby

# Data 
## 2. A description of the data and how it will be used to solve the problem. 

I will take postal codes, neigboorhood info, geo location from, **https://postal-codes.cybo.com/germany/berlin/#listcodes**, **https://postal-codes.cybo.com/netherlands/amsterdam/#listcodes** then using the foursquare API, I will obtain data about the venues in each neiborhood to compare them.

The details page on **cybo.com** have **Neighborhoods**, **Coordinates**, **area** and **population** fields 
Foursquare endpoint gives information about the venues **https://api.foursquare.com/v2/venues/explore**

First entry from the Berlin codes:


Postal Code|City|Administrative Region|Population|Area
 --- | --- | --- | --- | --- 
10115|Berlin|Berlin|16,678|2.421 km²

And in the details page **https://postal-codes.cybo.com/germany/10115_berlin-berlin/** fields that I will use:

| | |
:--- | --- 
**Neighborhoods**|Mitte
**Coordinates**|52.533706954560394° / 13.387223860255002°

Interesing information from the API are in the items, especialy venue fields.


Getting data from cybo.com in separate notebook, here I will import the obtained files:

In [56]:
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from IPython.core.display import display, HTML

In [2]:
def clean_df_codes(df: pd.DataFrame):
    df = df.replace(',','', regex=True)
    df['Population'] = pd.to_numeric(df['Population'], errors='coerce')
    df['Area km²'] = pd.to_numeric(df['Area km²'], errors='coerce')
    df.dropna(inplace=True)
    df['Population density per km²'] = df['Population']/df['Area km²']
    df['Population density per km²'] = df['Population density per km²'].round(2)
    df['Population'] = df['Population'].astype(int)
    return df

In [3]:
berlin_codes_df = pd.read_csv('berlin_codes.csv', delimiter=';')
berlin_codes_df = clean_df_codes(berlin_codes_df)
berlin_codes_df.head()

,Postal Code,City,Administrative Region,Population,Area km²,Population density per km²
0,10115,Berlin,Berlin,16678,2.421,6888.89
1,10117,Berlin,Berlin,24223,3.321,7293.89
2,10119,Berlin,Berlin,7408,0.857,8644.11
3,10178,Berlin,Berlin,14069,1.872,7515.49
4,10179,Berlin,Berlin,15897,2.183,7282.18


In [4]:
berlin_details_df = pd.read_csv('berlin_details.csv', delimiter=';', thousands=',')
berlin_details_df.head()

,Postal Code,Median Age,Neighborhoods,Latitude,Longitude
0,10115,43.0,Mitte,52.533707,13.387224
1,10117,43.0,Mitte,52.518746,13.390193
2,10119,43.0,"Bezirk Pankow, Mitte",52.532666,13.407149
3,10178,43.0,Mitte,52.523474,13.412203
4,10179,43.0,"Luisenstadt, Mitte",52.514591,13.419699


In [5]:
amsterdam_codes_df = pd.read_csv('amsterdam_codes.csv', delimiter=';')
amsterdam_codes_df = clean_df_codes(amsterdam_codes_df)
amsterdam_codes_df.rename(columns={'Postal District': 'Postal Code'}, inplace=True)
amsterdam_codes_df.head()

,Postal Code,City,Administrative Region,Population,Area km²,Population density per km²
0,1011,Amsterdam,North Holland,6606,1.032,6401.16
1,1012,Amsterdam,North Holland,7067,1.207,5855.01
2,1013,Amsterdam,North Holland,26792,6.300,4252.70
3,1014,Amsterdam,North Holland,15056,2.699,5578.36
4,1015,Amsterdam,North Holland,5926,0.776,7636.60


In [6]:
amsterdam_details_df = pd.read_csv('amsterdam_details.csv', delimiter=';', thousands=',')
amsterdam_details_df.head()

,Postal Code,Median Age,Neighborhoods,Latitude,Longitude
0,1011,36.3,"Amsterdam-Centrum, Stadsdeel Centrum",52.371124,4.903752
1,1012,36.3,"Amsterdam-Centrum, Centrum, De Wallen, Stadsde...",52.373179,4.894910
2,1013,36.3,"Amsterdam-Centrum, Amsterdam-West, Haarlemmerb...",52.387662,4.883396
3,1014,36.3,"Amsterdam-West, Bedrijventerrein Sloterdijk, S...",52.393066,4.853503
4,1015,36.3,"Amsterdam-Centrum, Grachtengordel-West, Jordaan",52.378205,4.882973


In [18]:
berlin_merged = pd.merge(berlin_codes_df, berlin_details_df, on='Postal Code', how='left')
berlin_merged.to_csv('berlin_regions.csv', sep=';')
berlin_merged.head()

,Postal Code,City,Administrative Region,Population,Area km²,Population density per km²,Median Age,Neighborhoods,Latitude,Longitude
0,10115,Berlin,Berlin,16678,2.421,6888.89,43.0,Mitte,52.533707,13.387224
1,10117,Berlin,Berlin,24223,3.321,7293.89,43.0,Mitte,52.518746,13.390193
2,10119,Berlin,Berlin,7408,0.857,8644.11,43.0,"Bezirk Pankow, Mitte",52.532666,13.407149
3,10178,Berlin,Berlin,14069,1.872,7515.49,43.0,Mitte,52.523474,13.412203
4,10179,Berlin,Berlin,15897,2.183,7282.18,43.0,"Luisenstadt, Mitte",52.514591,13.419699


In [17]:
amsterdam_merged = pd.merge(amsterdam_codes_df, amsterdam_details_df, on='Postal Code', how='left')
amsterdam_merged.to_csv('amsterdam_regions.csv', sep=';')
amsterdam_merged.head()

,Postal Code,City,Administrative Region,Population,Area km²,Population density per km²,Median Age,Neighborhoods,Latitude,Longitude
0,1011,Amsterdam,North Holland,6606,1.032,6401.16,36.3,"Amsterdam-Centrum, Stadsdeel Centrum",52.371124,4.903752
1,1012,Amsterdam,North Holland,7067,1.207,5855.01,36.3,"Amsterdam-Centrum, Centrum, De Wallen, Stadsde...",52.373179,4.894910
2,1013,Amsterdam,North Holland,26792,6.300,4252.70,36.3,"Amsterdam-Centrum, Amsterdam-West, Haarlemmerb...",52.387662,4.883396
3,1014,Amsterdam,North Holland,15056,2.699,5578.36,36.3,"Amsterdam-West, Bedrijventerrein Sloterdijk, S...",52.393066,4.853503
4,1015,Amsterdam,North Holland,5926,0.776,7636.60,36.3,"Amsterdam-Centrum, Grachtengordel-West, Jordaan",52.378205,4.882973


In [9]:
berlin_merged.describe()

,Postal Code,Population,Area km²,Population density per km²,Median Age,Latitude,Longitude
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,12262.685567,17621.417526,5.484062,5355.793144,43.095876,52.505112,13.393498
std,1287.341587,10837.421963,8.476452,2050.216635,0.559777,0.055279,0.116044
min,10115.000000,2869.000000,0.444000,277.140000,43.000000,52.363244,13.143126
25%,10823.500000,9730.250000,1.619250,4190.742500,43.000000,52.467486,13.317487
50%,12356.000000,15129.000000,2.952000,5876.970000,43.000000,52.504849,13.384287
75%,13354.500000,22677.500000,6.500000,6699.890000,43.000000,52.542604,13.460486
max,15562.000000,57275.000000,77.500000,8950.650000,48.000000,52.642215,13.799300


In [10]:
amsterdam_merged.describe()

,Postal Code,Population,Area km²,Population density per km²,Median Age,Latitude,Longitude
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,1060.358025,9830.283951,2.646741,5044.703704,36.403704,52.364022,4.892978
std,29.733949,5809.643077,2.612032,1851.930209,0.667354,0.029773,0.057825
min,1011.000000,878.000000,0.242000,175.110000,36.300000,52.293492,4.758401
25%,1034.000000,6128.000000,1.128000,4252.700000,36.300000,52.347895,4.853503
50%,1061.000000,8852.000000,1.880000,5398.920000,36.300000,52.362742,4.894881
75%,1083.000000,13174.000000,2.915000,6486.340000,36.300000,52.387662,4.935605
max,1114.000000,28120.000000,13.900000,7636.600000,42.300000,52.419563,5.016719


In [11]:
address_berlin = 'Berlin, Germany'
address_amsterdam = 'Amsterdam, Netherlands'

geolocator = Nominatim(user_agent="explorer")
berlin_location = geolocator.geocode(address_berlin)
berlin_lat, berlin_lon = (berlin_location.latitude, berlin_location.longitude)

amsterdam_location = geolocator.geocode(address_amsterdam)
amsterdam_lat, amsterdam_lon = (amsterdam_location.latitude, amsterdam_location.longitude)
print(f'The geograpical coordinate of Berlin are {berlin_lat}, {berlin_lon}. and Amsterdam are {amsterdam_lat}, {amsterdam_lon}')

The geograpical coordinate of Berlin are 52.5170365, 13.3888599. and Amsterdam are 52.3727598, 4.8936041


In [12]:
def get_map(df: pd.DataFrame, city_lat, city_lon, zoom=10):
    map = folium.Map(location=[city_lat, city_lon], zoom_start=zoom)

    # add markers to map
    for lat, lng, admin_region, neighborhoods in zip(df['Latitude'], df['Longitude'], df['Administrative Region'], df['Neighborhoods']):
        label = f'{neighborhoods}, {admin_region}'
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)  
        
    return map

In [34]:
berlin_map = get_map(berlin_merged, berlin_lat, berlin_lon, 11)
berlin_map

In [32]:
amsterdam_map = get_map(amsterdam_merged, amsterdam_lat, amsterdam_lon, 12)
amsterdam_map

In [19]:
berlin_venues = pd.read_csv('berlin_venues.csv', sep=';')
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")
berlin_onehot['Postal Code'] = berlin_venues['Neighborhood'] 

In [20]:
cols = list(berlin_onehot)
cols.insert(0, cols.pop(cols.index('Postal Code')))
berlin_onehot = berlin_onehot.loc[:, cols]
berlin_grouped = berlin_onehot.groupby('Postal Code').mean().reset_index()
berlin_grouped.head()

,Postal Code,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,10115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.0,0.0
1,10117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.030000,0.00,0.0,0.0,0.01,0.0,0.0
2,10119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.021978,0.00,0.0,0.0,0.00,0.0,0.0
3,10178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.00,0.0,0.0
4,10179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.0,0.0


In [21]:
amsterdam_venues = pd.read_csv('amsterdam_venues.csv', sep=';')
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")
amsterdam_onehot['Postal Code'] = amsterdam_venues['Neighborhood'] 

In [22]:
cols = list(amsterdam_onehot)
cols.insert(0, cols.pop(cols.index('Postal Code')))
amsterdam_onehot = amsterdam_onehot.loc[:, cols]
amsterdam_grouped = amsterdam_onehot.groupby('Postal Code').mean().reset_index()
amsterdam_grouped.head()

,Postal Code,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,1011,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.025,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,1012,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.02,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,1013,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.00,...,0.0,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.020000,0.0,0.0
3,1014,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,1015,0.0,0.0,0.0,0.010989,0.0,0.0,0.0,0.000,0.00,...,0.0,0.0,0.0,0.0,0.010989,0.010989,0.010989,0.010989,0.0,0.0


In [106]:
berlin_conds = berlin_merged[['Postal Code', 'Population density per km²', 'Median Age']]
amsterdam_conds = amsterdam_merged[['Postal Code', 'Population density per km²', 'Median Age']]

both_cities_conds = pd.concat([berlin_conds, amsterdam_conds])
print("Berlin df: ", berlin_conds.shape, "Amsterdam df: ", amsterdam_conds.shape, "both cities df: ", both_cities_conds.shape)

Berlin df:  (194, 3) Amsterdam df:  (81, 3) both cities df:  (275, 3)


In [107]:
from sklearn import preprocessing
both_cities_conds_id = both_cities_conds.set_index('Postal Code')
min_max_scaler = preprocessing.MinMaxScaler()
both_cities_conds = pd.DataFrame(min_max_scaler.fit_transform(both_cities_conds_id), columns=both_cities_conds_id.columns, index=both_cities_conds_id.index)
both_cities_conds = both_cities_conds.reset_index()
both_cities_conds.head()

,Postal Code,Population density per km²,Median Age
0,10115,0.765056,0.57265
1,10117,0.811207,0.57265
2,10119,0.965069,0.57265
3,10178,0.836459,0.57265
4,10179,0.809873,0.57265


In [28]:
# def return_most_common_venues(row, num_top_venues):
#     row_categories = row.iloc[1:]
#     row_categories_sorted = row_categories.sort_values(ascending=False)
    
#     return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
# num_top_venues = 10

# indicators = ['st', 'nd', 'rd']

# # create columns according to number of top venues
# columns = ['Postal Code']
# for ind in np.arange(num_top_venues):
#     try:
#         columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
#     except:
#         columns.append('{}th Most Common Venue'.format(ind+1))

# # create a new dataframe
# berlin_venues_sorted = pd.DataFrame(columns=columns)
# berlin_venues_sorted['Postal Code'] = berlin_grouped['Postal Code']

# for ind in np.arange(berlin_grouped.shape[0]):
#     berlin_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

# berlin_venues_sorted.head()

In [57]:
def show_maps_sidebyside(map1, map2):
    htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 49.7%; margin: 0 auto; border: 2px solid black"></iframe>'
            '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 49.7%; margin: 0 auto; border: 2px solid black"></iframe>'
            .format(map1.get_root().render().replace('"', '&quot;'),500,500,
                    map2.get_root().render().replace('"', '&quot;'),500,500))
    display(htmlmap)
 

In [66]:
%%capture --no-display
show_maps_sidebyside(berlin_map, amsterdam_map)